In [1]:
%load_ext autoreload
%autoreload 2

from pandas import read_excel
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import random
import timeit

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from modAL.models import ActiveLearner
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.cross_decomposition import PLSRegression

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem,DataStructs,Draw,PandasTools,Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger

from IPython.display import Image

# Configure the logging - RDKit is rather verbose..
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
# Set the molecule representation to be SVG 
PandasTools.molRepresentation='svg'
# Loading and standardization method for SMILES -> RDKit molecule object
uncharger = rdMolStandardize.Uncharger()


# Loading and standardization method for SMILES -> RDKit molecule object
uncharger = rdMolStandardize.Uncharger()

import sys
sys.path.insert(0, '/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/')
import functions as fun

In [8]:
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem,DataStructs,Draw,PandasTools,Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
from rdkit import RDLogger

from IPython.display import Image

# Configure the logging - RDKit is rather verbose..
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)
# Set the molecule representation to be SVG 
PandasTools.molRepresentation='svg'
# Loading and standardization method for SMILES -> RDKit molecule object
uncharger = rdMolStandardize.Uncharger()


# Loading and standardization method for SMILES -> RDKit molecule object
uncharger = rdMolStandardize.Uncharger()

In [2]:
# Loading data sets
specs = pd.read_csv('/home/jovyan/covid-data/SPECS_ready_POLINA.csv')
sss_batchA = pd.read_csv('/home/jovyan/covid-data/COVID_BatchA_ready_POLINA.csv')

#Creating final dataset
data = pd.merge(specs[['ID_library', 'smiles']], sss_batchA[['ID_covid', 'Intensity_MeanIntensity_illumMITO_cells.1','Labels']], 
                left_on= 'ID_library', right_on='ID_covid', how='left')
data = data.drop(['ID_covid'],axis=1)
data.head() 

,ID_library,smiles,Intensity_MeanIntensity_illumMITO_cells.1,Labels
0,CBK000155,C[C@H](Cc1ccc(OCC([O-])=O)cc1)NC[C@H](O)c1cccc...,0.010426,0
1,CBK000156,CC(C)(C)NCC(O)COc1cccc2[nH]c(=O)[nH]c12,0.009417,0
2,CBK000161,[H][C@@](CC)(NC(C)C)[C@]([H])(O)c1ccc(O)c2[nH]...,0.008106,0
3,CBK000162,CC(C)N[C@@H](C)[C@@H](O)COc1ccc(C)c2CCCc12,0.010586,0
4,CBK000164,CC(C)NCC(O)COc1ccc(COCCOC(C)C)cc1,0.010333,0


In [3]:
data.groupby('Labels').count()

,ID_library,smiles,Intensity_MeanIntensity_illumMITO_cells.1
Labels,,,
0,5121,5121,5121
1,51,51,51


In [4]:
# First we need to parse the SMILES and create RDKit 'molecule' objects 
# the 'standardize' function was the one we defined in the first cell with code in this file
data['MOL'] = data['smiles'].apply(fun.standardize)
data.head()

### There is a problem with some of the smiles as they return none when trying to convert to molecule. 
### The next step is to fix those cases which may be a product of a typo. 

# Looking for None values
corrupted_smiles_idx_list = data[data['MOL'].isnull()].index.tolist() #finding None values
data[data['MOL'].isnull()].head(5)

,ID_library,smiles,Intensity_MeanIntensity_illumMITO_cells.1,Labels,MOL
5164,CBK310016,Cc1nc2c(c(Cl)c(Cl)cc2n1Cc1ccc(cc1)B12OC(=O)CN1...,0.009946,0,None


In [5]:
data = data.drop([5164])
print(f"There are {len(data)} rows in data")

There are 5171 rows in data


In [6]:
data[data['MOL'].isnull()]

,ID_library,smiles,Intensity_MeanIntensity_illumMITO_cells.1,Labels,MOL


In [7]:
data = data.drop(['MOL'],axis=1)

#Creating molecue objects again
data['MOL'] = data['smiles'].apply(fun.standardize)
data.head()

,ID_library,smiles,Intensity_MeanIntensity_illumMITO_cells.1,Labels,MOL
0,CBK000155,C[C@H](Cc1ccc(OCC([O-])=O)cc1)NC[C@H](O)c1cccc...,0.010426,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
1,CBK000156,CC(C)(C)NCC(O)COc1cccc2[nH]c(=O)[nH]c12,0.009417,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
2,CBK000161,[H][C@@](CC)(NC(C)C)[C@]([H])(O)c1ccc(O)c2[nH]...,0.008106,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
3,CBK000162,CC(C)N[C@@H](C)[C@@H](O)COc1ccc(C)c2CCCc12,0.010586,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."
4,CBK000164,CC(C)NCC(O)COc1ccc(COCCOC(C)C)cc1,0.010333,0,"<svg xmlns=""http://www.w3.org/2000/svg"" xmlns:..."


In [9]:
# Check if any of the SMILES contain multiple fragments - 
# this can sometimes cause issues when computing descriptors
data['FRAGS'] =  data['MOL'].apply(lambda mol: len(Chem.GetMolFrags(mol,asMols=True)))
print("Fragments: {0}".format(data['FRAGS'].nunique()))

Fragments: 1


In [31]:
for radio in np.arange(3,16,1):
    morgan_radius = int(radio)
    morgan_n_bits = 2048
    # Feature creation
    X_morgan, _, y = fun.feature_creation(morgan_radius, morgan_n_bits, fp_n_bits, data)
    full_data_BatchA = pd.DataFrame(X_morgan)
    full_data_BatchA['Label'] = y
    print('Radio: {}. Number of unique vectors:{}'.format(radio,len(full_data_BatchA.value_counts())))

Radio: 3. Number of unique vectors:5143
Radio: 4. Number of unique vectors:5144
Radio: 5. Number of unique vectors:5146
Radio: 6. Number of unique vectors:5149
Radio: 7. Number of unique vectors:5149
Radio: 8. Number of unique vectors:5150
Radio: 9. Number of unique vectors:5151
Radio: 10. Number of unique vectors:5152
Radio: 11. Number of unique vectors:5153
Radio: 12. Number of unique vectors:5153
Radio: 13. Number of unique vectors:5153
Radio: 14. Number of unique vectors:5153
Radio: 15. Number of unique vectors:5153


In [12]:
# Parameters for features
morgan_radius = 3
morgan_n_bits = 2048
fp_n_bits = 2048

# Feature creation
X_morgan, _, y = fun.feature_creation(morgan_radius, morgan_n_bits, fp_n_bits, data)

full_data_BatchA_mgn = pd.DataFrame(X_morgan)
full_data_BatchA_mgn['Label'] = y
print('Number of unique vectors:{}'.format(len(full_data_BatchA_mgn.value_counts())))

Number of unique vectors:5143


In [21]:
print('Number of repeated row whos label is 0: \n{}'.format((full_data_BatchA_mgn[full_data_BatchA_mgn.duplicated(keep=False)]['Label']==0).count()))

Number of repeated row whos label is 0: 
50


In [31]:
full_data_BatchA_mgn=full_data_BatchA_mgn.drop_duplicates(keep='first')
print('full_data_BatchA_mgn has {} rows of which {} are unique'.format(len(full_data_BatchA_mgn),len(full_data_BatchA_mgn.value_counts())))

full_data_BatchA_mgn has 5143 rows of which 5143 are unique


So, given that all labels are 0 we can just keep one of the repetitions to keep everything unique

In [33]:
name = r'/home/jovyan/Thesis_ActLearn_DOP_2022/main/active_learning/data/full_data_BatchA.csv'
full_data_BatchA_mgn.to_csv(name, sep=',', encoding='utf-8',index=False)